In [20]:
!pip install spacy

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/13.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/13.9 MB 11.0 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.9 MB 10.7 MB/s eta 0:00:01
   ------------------ --------------------- 6.6/13.9 MB 11.1 MB/s eta 0:00:01
   -------------------------- ------------- 9.2/13.9 MB 11.3 MB/s eta 0:00:01
   --------------------------------- ------ 11.5/13.9 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------  13.9/13.9 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 13.9/

In [1]:
import spacy
from spacy.matcher import PhraseMatcher
import re

In [2]:
!pip install en_core_web_sm-3.8.0-py3-none-any.whl

Processing d:\clony\doc_load_automation\en_core_web_sm-3.8.0-py3-none-any.whl


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'D:\\clony\\Doc_Load_Automation\\en_core_web_sm-3.8.0-py3-none-any.whl'



In [3]:
!pip install en_core_web_md-3.8.0-py3-none-any.whl

Processing d:\clony\doc_load_automation\en_core_web_md-3.8.0-py3-none-any.whl


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'D:\\clony\\Doc_Load_Automation\\en_core_web_md-3.8.0-py3-none-any.whl'



In [4]:
nlp = spacy.load("en_core_web_md")
#text = "Pinarayi Vijayan, Chief Minister of Kerala, inaugurated the second phase of Kochi Metro on 15 March 2021 with a budget of ₹500 crore."


In [5]:
text ="Bid Details Bid End Date Time 20-02-2025 15:00:00 Bid Opening Date Time 20-02-2025 15:30:00 Bid Offer Validity (From End Date) ( ) 90 (Days) Ministry State Name ! Ministry Of Housing & Urban Affairs (mohua) Department Name Na Organisation Name Kochi Metro Rail Limited Office Name % Kochi Metro Rail Limited & Buyer Email mmt.kmrl@kerala.gov.in Item Category Financial Audit Services - As per ATC; Audit Firm Contract Period 3 Year(s) MSE Exemption for Years of Experience and Turnover , No Startup Exemption for Years of Experience and Turnover , 0 % No Document required from seller & 0 Certificate (Requested in ATC),Additional Doc 1 (Requested in ATC),Additional Doc 2 (Requested in ATC),Additional Doc 3 (Requested in ATC),Additional Doc 4 (Requested in ATC) In case any bidder is seeking exemption from Experience Turnover Criteria, the supporting documents to prove his eligibility for exemption must be uploaded for evaluation by the buyer Do you want to show documents uploaded by bidders to', 'ion from Experience Turnover Criteria, the supporting documents to prove his eligibility for exemption must be uploaded for evaluation by the buyer Do you want to show documents uploaded by bidders to all bidders participated in bid? Yes Bid to RA enabled 2 % 4& 4 No Type of Bid 5 Two Packet Bid Time allowed for Technical Clarifications during technical evaluation 6 0 8 7 Days Evaluation Method 6 9 Total value wise evaluation Bid Number & ( : ): GEM 2025 B 5848801 Dated 4 : 30-01-2025 Bid Document 0 1 5 ys fe GeM"

In [6]:
def get_deadline(dates,docs):
    deadline_keywords = ["deadline", "due", "by", "before", "expires", "submission","submit by","deadline",
                         "due date","last date","cut-off date","closing date","final date","end date",
                         "last date for submission","due for submission","to be submitted by","bid submission end date",
                         "tender submission date","online submission deadline","closing time for submission",
                         "application deadline","application closing date","last date for applying","proposal due date",
                         "proposal submission deadline","RFP submission date","EOI submission date (Expression of Interest)",
                         "tender closing date","bid closing date","last date of receipt of bids","last date of receipt of tenders",
                         "shall not be accepted after","no application will be entertained after","valid till","to reach by",
                         "on or before","time limit for submission","period ends on","Bid End Date/Time"]
    
    matcher = PhraseMatcher(nlp.vocab,attr="LOWER")
    pattern = [nlp(keyword) for keyword in deadline_keywords]
    matcher.add("DEADLINE_KEYWORD",pattern)
    for date in dates:
        if hasattr(date, "start"):
            date_start = date.start
            date_text = date.text
            sentence = date.sent
        else:
            start_pos = text.find(str(date))
            end_pos = start_pos + len(str(date))
            span = docs.char_span(start_pos, end_pos)
            if span is not None:
                date_start = span.start
                date_text = str(date)
                sentence = span.sent
            else:
                # Skip if no valid span found
                continue

        for match_id, start, end in matcher(docs):
            if abs(start - date_start) <= 10:
                print(f"Deadline-related phrase: {sentence.text}")

def get_financial_details(entities,docs):
    for ent in entities:
        if hasattr(ent, "start"):
            date_start = ent.start
            date_text = ent.text
            sentence = ent.sent
        else:
            start_pos = text.find(str(ent))
            end_pos = start_pos + len(str(ent))
            span = docs.char_span(start_pos, end_pos)
            if span is not None:
                date_start = span.start
                date_text = str(ent)
                sentence = span.sent

        print(f"{sentence.text}")

docs = nlp(text)
spacy_dates = [ent.text for ent in docs.ents if ent.label_ == 'DATE']
spacy_money = [ent.text for ent in docs.ents if ent.label_ == 'MONEY']

# 2. Regex dates (custom pattern for Numeric+Time format)
regex_dates = re.findall(r"\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}", text)

# Also capture simpler dates if needed:
simple_regex_dates = re.findall(r"\d{2}-\d{2}-\d{4}", text)

# 3. Combine all detected dates (avoid duplicates)
all_dates = set(spacy_dates + regex_dates + simple_regex_dates)

get_deadline(all_dates,docs)
get_financial_details(spacy_money,docs)

Deadline-related phrase: Bid Details Bid End Date Time 20-02-2025 15:00:00 Bid Opening Date Time 20-02-2025 15:30:00 Bid Offer Validity (From End Date) ( ) 90 (Days) Ministry State Name !
Deadline-related phrase: Bid Details Bid End Date Time 20-02-2025 15:00:00 Bid Opening Date Time 20-02-2025 15:30:00 Bid Offer Validity (From End Date) ( ) 90 (Days) Ministry State Name !
